In [17]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("./유방암.csv", engine='python')
data.head()

,diagnosis,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
data.isnull().sum(axis = 0)

diagnosis                  0
mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
dtype: int64

In [16]:
table = str.maketrans(' ', '_')
rename_dict = { col:col.upper().translate(table)  for col in data.columns}

data.rename(columns=rename_dict,inplace=True)
data.head()

,DIAGNOSIS,MEAN_RADIUS,MEAN_TEXTURE,MEAN_PERIMETER,MEAN_AREA,MEAN_SMOOTHNESS,MEAN_COMPACTNESS,MEAN_CONCAVITY,MEAN_CONCAVE_POINTS,MEAN_SYMMETRY,MEAN_FRACTAL_DIMENSION,RADIUS_ERROR,TEXTURE_ERROR,PERIMETER_ERROR,AREA_ERROR,SMOOTHNESS_ERROR,COMPACTNESS_ERROR,CONCAVITY_ERROR,CONCAVE_POINTS_ERROR,SYMMETRY_ERROR,FRACTAL_DIMENSION_ERROR,WORST_RADIUS,WORST_TEXTURE,WORST_PERIMETER,WORST_AREA,WORST_SMOOTHNESS,WORST_COMPACTNESS,WORST_CONCAVITY,WORST_CONCAVE_POINTS,WORST_SYMMETRY,WORST_FRACTAL_DIMENSION
0,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [24]:
np.random.seed(seed=1234)
msk = np.random.rand(data.shape[0],) < 0.7
df_train = data[msk]
df_test = data[~msk]

In [25]:
df_train = sm.add_constant(df_train)

/home/donghyun-lee/miniconda3/envs/DS/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [37]:
'+'.join(df_train.columns[2:])

'MEAN_RADIUS+MEAN_TEXTURE+MEAN_PERIMETER+MEAN_AREA+MEAN_SMOOTHNESS+MEAN_COMPACTNESS+MEAN_CONCAVITY+MEAN_CONCAVE_POINTS+MEAN_SYMMETRY+MEAN_FRACTAL_DIMENSION+RADIUS_ERROR+TEXTURE_ERROR+PERIMETER_ERROR+AREA_ERROR+SMOOTHNESS_ERROR+COMPACTNESS_ERROR+CONCAVITY_ERROR+CONCAVE_POINTS_ERROR+SYMMETRY_ERROR+FRACTAL_DIMENSION_ERROR+WORST_RADIUS+WORST_TEXTURE+WORST_PERIMETER+WORST_AREA+WORST_SMOOTHNESS+WORST_COMPACTNESS+WORST_CONCAVITY+WORST_CONCAVE_POINTS+WORST_SYMMETRY+WORST_FRACTAL_DIMENSION'

In [39]:
log_model = sm.Logit.from_formula("DIAGNOSIS~"+"+".join(df_train.columns[2:]) + "+1",df_train)

In [42]:
log_res = log_model.fit(method="bfgs")
print(log_res.summary())

         Current function value: 0.081660
         Iterations: 35
         Function evaluations: 47
         Gradient evaluations: 40
                           Logit Regression Results                           
Dep. Variable:              DIAGNOSIS   No. Observations:                  385
Model:                          Logit   Df Residuals:                      354
Method:                           MLE   Df Model:                           30
Date:                Mon, 04 Feb 2019   Pseudo R-squ.:                  0.8774
Time:                        22:39:19   Log-Likelihood:                -31.439
converged:                      False   LL-Null:                       -256.48
                                        LLR p-value:                 1.926e-76
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.8465     26.711     

/home/donghyun-lee/miniconda3/envs/DS/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/home/donghyun-lee/miniconda3/envs/DS/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/home/donghyun-lee/miniconda3/envs/DS/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [44]:
y_pred = log_res.predict(df_test)
y_pred_class = (y_pred > 0.5).astype(int)
print("Accuracy: {0:.3f}".format(metrics.accuracy_score(df_test["DIAGNOSIS"],y_pred_class)))

Accuracy: 0.957
